# Purpose

 Train neural netwok to forecast horror author based on phrase

## 0. Imports

In [148]:
import numpy as np
import pandas as pd

import spacy
from nltk.corpus import stopwords
import string
from collections import Counter

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

## 1. Preprocessing

Dataset consist of phrases of 3 horror authors: Edgar Po (EAP), HP Lovecraft (HPL), Mary Shelley (MWS)

Dataset source - https://www.kaggle.com/competitions/spooky-author-identification/data

In [149]:
dataset = pd.read_csv('train.csv')

In [150]:
dataset.head(3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [151]:
# OneHotEncoding for y 
one_hot = pd.get_dummies(dataset['author'])
one_hot.head(3)

,EAP,HPL,MWS
0,1,0,0
1,0,1,0
2,1,0,0


In [152]:
X = dataset['text'].values
y = dataset.join(one_hot)[['EAP','HPL','MWS']].values

In [153]:
# set function for clear text from personal pronoun, punctuation, stopwords by spaCy
nlp = spacy.load('en_core_web_sm')
punctuations = string.punctuation

def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [154]:
stopwords = stopwords.words('english')

In [155]:
# clear text and check for completeness
print('Original training data shape: ', X.shape)
train_cleaned = cleanup_text(X, logging=True)
print('Cleaned up training data shape: ', train_cleaned.shape)

Original training data shape:  (19579,)
Processed 1000 out of 19579 documents.
Processed 2000 out of 19579 documents.
Processed 3000 out of 19579 documents.
Processed 4000 out of 19579 documents.
Processed 5000 out of 19579 documents.
Processed 6000 out of 19579 documents.
Processed 7000 out of 19579 documents.
Processed 8000 out of 19579 documents.
Processed 9000 out of 19579 documents.
Processed 10000 out of 19579 documents.
Processed 11000 out of 19579 documents.
Processed 12000 out of 19579 documents.
Processed 13000 out of 19579 documents.
Processed 14000 out of 19579 documents.
Processed 15000 out of 19579 documents.
Processed 16000 out of 19579 documents.
Processed 17000 out of 19579 documents.
Processed 18000 out of 19579 documents.
Processed 19000 out of 19579 documents.
Cleaned up training data shape:  (19579,)


In [156]:
# tokenezation and vectorization text data for training of model
vocab_size = 25000 # максимум уникальных слов для использования
max_len = 250 # максимальная длина предложения 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_cleaned)
len(tokenizer.word_index)

19374

In [157]:
tokenized_train = tokenizer.texts_to_sequences(train_cleaned)

In [158]:
X = pad_sequences(tokenized_train,maxlen=max_len)

In [159]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

print('X_train size: {}'.format(X_train.shape))
print('X_test size: {}'.format(X_test.shape))
print('y_train size: {}'.format(y_train.shape))
print('y_test size: {}'.format(y_test.shape))

X_train size: (15663, 250)
X_test size: (3916, 250)
y_train size: (15663, 3)
y_test size: (3916, 3)


## 2. Training of neural network

Recurrent neural networks based on LSTM (Long short-term memory) is more appropriate for such tasks, so select that type network

In [160]:
emd_dim = 32
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,emd_dim,input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=True)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(3, activation='sigmoid')
])

In [161]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [162]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 250, 32)           800000    
                                                                 
 bidirectional_5 (Bidirectio  (None, 250, 100)         33200     
 nal)                                                            
                                                                 
 global_max_pooling1d_5 (Glo  (None, 100)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 50)                5050      
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                

In [163]:
# model training
batch_size = 264
epochs = 5
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/5
48/48 [==============================] - 202s 4s/step - loss: 1.0750 - accuracy: 0.4093 - val_loss: 1.0158 - val_accuracy: 0.4858
Epoch 2/5
48/48 [==============================] - 194s 4s/step - loss: 0.7692 - accuracy: 0.6621 - val_loss: 0.6219 - val_accuracy: 0.7702
Epoch 3/5
48/48 [==============================] - 199s 4s/step - loss: 0.3656 - accuracy: 0.8665 - val_loss: 0.5652 - val_accuracy: 0.7913
Epoch 4/5
48/48 [==============================] - 199s 4s/step - loss: 0.2203 - accuracy: 0.9226 - val_loss: 0.5595 - val_accuracy: 0.8002
Epoch 5/5
48/48 [==============================] - 199s 4s/step - loss: 0.1364 - accuracy: 0.9540 - val_loss: 0.6638 - val_accuracy: 0.7929


In [164]:
# result
print("Training accuracy: %.2f%% / Validation accuracy: %.2f%%" % 
      (100*history.history['accuracy'][-1], 100*history.history['val_accuracy'][-1]))

Training accuracy: 95.40% / Validation accuracy: 79.29%


## 3. Check model on test

In [165]:
# prediction on test
predicted_prob = model.predict(X_test)
print(predicted_prob.shape)

123/123 [==============================] - 6s 42ms/step
(3916, 3)


In [166]:
# accuracy on test
test_accuracy = tf.keras.metrics.BinaryAccuracy()
test_accuracy.update_state(y_test, predicted_prob,
               sample_weight=None)

print("Test accuracy: %.2f%% " % 
      (100*test_accuracy.result().numpy()))

Test accuracy: 83.59% 


## Conclusion:
Accuracy on test at good level. Recurrent neural networks based on LSTM is efficient for that task.